# Классификация текстов с использованием Наивного Байесовского Классификатора

## Задание 1 (1 балл)

Откройте данные. Узнайте, сколько в них спам- и не спам-писем. Визуализируйте полученные соотношение подходящим образом.

In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import GridSearchCV

In [3]:
# откройте данные: ваш код здесь
df = pd.read_csv('data/spam_or_not_spam.csv', sep=',')
df.head()


,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   email   2999 non-null   object
 1   label   3000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


In [5]:
# рассчитайте частоты для классов : ваш код здесь
df.groupby('label').count()

,email
label,
0,2500
1,499


In [6]:
# визуализируйте результат: ваш код здесь
class_labels = {0: 'Не спам', 1: 'Спам'}
counts = df['label'].value_counts()
counts.index = counts.index.map(class_labels)
fig = go.Figure(data=[go.Pie(labels=counts.index, values=counts.values, hole=0.5)])
fig.update_layout(title="Распределение классов")
fig.show()

## Задание 2 (2 балла)

Вам необходимо предобработать ваши данные и перевести их в векторный вид. Подгрузим необходимый модуль:

In [7]:
from sklearn.feature_extraction.text import CountVectorizer


Замените в данных все пустые строки и строки, состоящие из пробелов, на пропуски (NaN). После этого удалите из данных все строки, в которых наблюдаются пропущенные значения.

In [8]:
#ваш код здесь
df.replace(['', ' '], np.nan, inplace=True)
df.dropna(inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2997 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   email   2997 non-null   object
 1   label   2997 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 70.2+ KB


Переводим данные в векторный вид:

In [9]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df["email"])

Определите, сколько теперь признаков в нашем наборе данных:

In [10]:
#ваш код здесь
X.shape[1]

34116

## Задание 3 (2 балла)

Определите целевую переменную и признаки:

In [11]:
#ваш код здесь
y = df['label']
# выводим результат
print(f'Матрица X: {X.shape}')
print(f'Метки классов y: {y.shape}')

Матрица X: (2997, 34116)
Метки классов y: (2997,)


Разделите выборку на обучающую и тестовую, используя стратифицированное разбиение (параметр `stratify` установите в значение вектора ответов y) размер тестовой выборки (`test_size`) возьмите как 0.25, параметр `random_state` определите со значением 42:

In [12]:
#ваш код здесь
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

print(f'Размер обучающей выборки: {X_train.shape}')
print(f'Размер тестовой выборки: {X_test.shape}')

Размер обучающей выборки: (2247, 34116)
Размер тестовой выборки: (750, 34116)


Рассчитайте среднее значение целевой переменной по тестовой выборке:

In [13]:
#ваш код здесь
mean_y_test = round(y_test.mean(), 3)
mean_y_test

0.165

## Задание 4 (3 балла)

Определите и обучите подходящий алгоритм с параметром alpha = 0.01

In [14]:
#ваш код здесь
clf = MultinomialNB(alpha=0.01)

clf.fit(X_train, y_train)

MultinomialNB(alpha=0.01)

Оцените результат с точки зрения всех известных вам метрик (не менее трёх):

In [15]:
#ваш код здесь
accuracy = clf.score(X_test, y_test)
print("Accuracy:", round(accuracy, 3))

y_pred = clf.predict(X_test)
precision = precision_score(y_test, y_pred)
print("Precision:", round(precision, 3))

recall = recall_score(y_test, y_pred)
print("Recall:", round(recall, 3))

f1 = f1_score(y_test, y_pred)
print("F1-мера:", round(f1, 3))

Accuracy: 0.987
Precision: 1.0
Recall: 0.919
F1-мера: 0.958


Нарисуйте ROC-кривую:

In [16]:
#ваш код здесь
fpr, tpr, thresholds = roc_curve(y_test, clf.predict_proba(X_test)[:, 1])

auc = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
print("AUC-ROC:", auc)

fig = go.Figure()
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name='ROC-кривая (AUC-ROC = %0.2f)' % auc))
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', line=dict(dash='dash'), name='Случайная модель'))
fig.update_layout(xaxis_title='Ложно положительные решения', yaxis_title='Истинно положительные решения', title='ROC-кривая')
fig.show()

AUC-ROC: 0.9950659589817583


## Задание 5 (3 балла)

Переберите несколько значений alpha с помощью кросс-валидации. Оцените, зависит ли от этого параметра качество классификации.

In [17]:
param_grid = {'alpha': np.arange(0.01, 10, 0.05)}

grid_search = GridSearchCV(MultinomialNB(), param_grid, cv=5)

grid_search.fit(X_train, y_train)

best_alpha = grid_search.best_params_['alpha']
best_score = grid_search.best_score_
print("Наилучшее значение параметра alpha:", best_alpha)
print("Качество классификации:", best_score)

alphas = [params['alpha'] for params in grid_search.cv_results_['params']]
f1_scores = grid_search.cv_results_['mean_test_score']

fig = go.Figure()
fig.add_trace(go.Scatter(x=alphas, y=f1_scores, mode='markers', marker=dict(color='blue'), name='F1-мера'))

fig.add_annotation(x=best_alpha, y=best_score, text='Наилучшее значение alpha', showarrow=True, arrowhead=1, font=dict(color='red'))
fig.update_layout(xaxis_title='Значение параметра alpha', yaxis_title='Значение F1-меры', title='Зависимость F1-меры от параметра alpha')
fig.show()

Наилучшее значение параметра alpha: 0.11
Качество классификации: 0.9915456570155902
